In [3]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import style
style.use('ggplot')
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
#from hmmlearn import hmm
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.model_selection import KFold
import os
import openpyxl
from bs4 import BeautifulSoup
import re

In [2]:
class Closed:
    attrs = ['P', 'PART', 'ADV', 'CONJ', 'CONN', 'PRON', 'fragment', 'REL', 'PRV', 'POST', 'NEG', 'INDEF', 'Q', 'DEM', 'POSS', 'NUM']
    def __init__(self):
        self.adverb = {i.split()[0]: i.split()[1] for i in
                           open('../closed_class/Closed_lists_adverbs.txt').read().split('\n') if i != ''}
        self.compliment = {i.split()[0]: i.split()[1] for i in
                           open('../closed_class/Closed_lists_complementizers.txt').read().split('\n') if i != ''}
        self.conjunct = {i.split()[0]: i.split()[1] for i in
                           open('../closed_class/Closed_lists_conjunctions.txt').read().split('\n') if i != ''}
        self.connector = {i.split()[0]: i.split()[1] for i in
                           open('../closed_class/Closed_lists_connectors.txt').read().split('\n') if i != ''}
        self.preverb = {i.split()[0]: i.split()[1] for i in
                           open('../closed_class/Closed_lists_preverbs.txt').read().split('\n') if i != ''}
        self.pronoun = {i.split()[0]: i.split()[1] for i in
                           open('../closed_class/Closed_lists_pronouns.txt').read().split('\n') if i != ''}
        self.q_word = {i.split()[0]: i.split()[1] for i in
                           open('../closed_class/Closed_lists_q-words.txt').read().split('\n') if i != ''}
        self.fragment = {i.split()[0]: i.split()[1] for i in
                       open('../closed_class/Closed_lists_fragment.txt').read().split('\n') if i != ''}

In [3]:
# делаем обучающий сет

closed = Closed()
# считываем класс закрытых классов
speach_class = {i.split()[1]: i.split()[0] for i in open('../class.txt').read().split('\n') if i != ''}
# считываем части речи с их порядковым номером в словарь
features = open('../features.txt').read().split()
# считываем признаки

train_word_list = []  # массив для обучающих слов
train_features_set = []  # массив для обучающих признаков
train_tag_list = []  # массив с правильными частями речи

fname_list = os.listdir('../materials_xlsx/') # массив имён файлов с обучающими материалами в .xlsx

for fname in fname_list: # для каждого файла

    print('processing ' + fname + '\n')
    #os.system('libreoffice --convert-to xml --outdir ./materials_xml/ ../materials_xlsx/' + fname) # конвертируем в .xml
    # результат положится в папку materials_xml в данной дериктории

    soup = BeautifulSoup(open('./materials_xml/' + fname.replace('xlsx', 'xml')), 'xml') # читаем дерево

    row_list = []
    for l in soup.find_all('table', attrs={'table:name': 'Word Forms'}): # идём во второй лист "Word Forms"
        for i, line in enumerate(l.find_all('table-row')):
            row_list.append(re.split('\n{1,3}', line.get_text().replace('\xa0', '').strip('\n'))) #считываем каждый ряд
            # и получаем массив, где каждый элемент -- массив с содержимым каждой ячейки в этом ряду

    row_list.pop(0) # удаляем первые два ряда,
    row_list.pop(0) # потому что в них названия столбцов

    for row in row_list:
        if row[0] != '':
            if len(row) > 8 and row[8].strip() in speach_class.keys():
                example = []
                train_word_list.append(row[5].strip('!: ')) # добавляем слово
                for feat in features: # добавляем его признаки
                    if row[5].strip('!: ').endswith(feat):
                        example.append(1.0) # есть признак
                    else:
                        example.append(0.0) # нет признака
                example.append(float(row[10])) # добавляем номер в предложении
                train_tag_list.append(speach_class[row[8].strip()]) # порядковый номер правильной части речи
                train_features_set.append(example) # получаем матрицу из примеров

X = np.array(train_features_set)
y = np.array(train_tag_list)

print('train_matrix made' + '\n')

processing Letters-NH-3_KUB-19-5-KBo-19-79.xlsx

processing Letters-NH-4_KUB-14-3.xlsx

train_matrix made



In [30]:
def get_estimation(algorithm, X_train, X_test, y_train, y_test):
    # считаем всякие оценки
    algorithm.fit(X_train, y_train)
    y_pred = algorithm.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    F1 = f1_score(y_test, y_pred, average='weighted')

    return [accuracy, precision, recall, F1]

In [17]:
def knn_pred(n, X_train, X_test, y_train, y_test):
    # k ближайших соседей
    knn = KNeighborsClassifier(n_neighbors=n, metric='euclidean')
    knn_estimation = get_estimation(knn, X_train, X_test, y_train, y_test)
    return knn_estimation

In [38]:
def lr_pred(с, X_train, X_test, y_train, y_test):
    # логистическая регрессия
    lr = LogisticRegression(penalty="l2", fit_intercept=True, max_iter=100, C=с, solver="lbfgs")
    lr_estimation = get_estimation(lr, X_train, X_test, y_train, y_test)
    return lr_estimation

In [44]:
def svm_pred(c, X_train, X_test, y_train, y_test):
    # метод опорных векторов
    svm = SVC(C=c)
    svm_estimation = get_estimation(svm, X_train, X_test, y_train, y_test)
    return svm_estimation

In [21]:
def kfolds_validation(model, param, X, y):
    # кросс-валидация
    kf = KFold(n_splits=4, shuffle=True, random_state=12345)
    estimations = []

    for train, test in kf.split(X):
        X_train, X_test, y_train, y_test = X[train], X[test], y[train], y[test]
        estimations.append(model(param, X_train, X_test, y_train, y_test)[3])

    estimation = np.array(estimations).mean() # средняя оценка по фолдам

    return estimation

In [20]:
def get_params_dic(model, params, X, y):
    # создаём словарь, где ключи -- параметр, а значения -- оценка при этом параметре
    estimations = {}

    for param in params:
        estimations[param] = kfolds_validation(model, param, X, y)

    return estimations

In [19]:
def create_plot(model, estimation):
    # строим график зависимости параметра и оценки
    plt.plot([key for key in estimation.keys()], [value for value in estimation.values()])
    plt.title(model + ' F1-score')
    plt.xlabel('Hyperparameters')
    plt.ylabel('F1-score')
    plt.tight_layout()
    plt.savefig('./graphs/' + model + ' F1-score')
    plt.close()
    print(model + ' plot created\n')

In [46]:
ns = np.arange(1, 150, 10)  # количество соседей
cs_lr = np.logspace(-2, 10, 8, base=10)  # параметр регуляризации для логрега
cs_svm = [0.01, 0.03, 0.1, 0.3, 1, 3, 10, 30, 60, 90] # параметр регуляризации для опорных векторов

In [31]:
knn_estimation = get_params_dic(knn_pred, ns, X, y)
print('knn F1-score: ' + '\n', '\n'.join(['{}: {}'.format(key, knn_estimation[key]) for key in knn_estimation]), '\n')
create_plot('knn algorithm', knn_estimation)

/home/sokureo/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/sokureo/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/sokureo/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1115: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/home/sokureo/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1115: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


knn accuracy: 
 1: 0.6478423879850923
11: 0.6417157368231654
21: 0.6112645330406992
31: 0.5880382375474585
41: 0.5611931414061432
51: 0.563153641694421
61: 0.5842704050113614
71: 0.6045543373438369
81: 0.588100201033911
91: 0.5819309844064607
101: 0.5836251284935134
111: 0.589430333268643
121: 0.5946708295945445
131: 0.6044231171719052
141: 0.6097420152625423 

knn algorithm plot created



In [34]:
lr_estimation = get_params_dic(lr_pred, cs_lr, X, y)
print('log reg F1-score: ' + '\n', '\n'.join(['{}: {}'.format(key, lr_estimation[key]) for key in lr_estimation]), '\n')
create_plot('log regression algorithm', lr_estimation)

/home/sokureo/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/sokureo/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/sokureo/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1115: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/home/sokureo/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1115: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


log reg F1-score: 
 0.01: 0.6135133160955508
0.517947467923121: 0.6995614815149891
26.826957952797247: 0.7124883138514506
1389.4954943731361: 0.7083285695408699
71968.56730011514: 0.7083285695408699
3727593.720314938: 0.7089621363003767
193069772.88832456: 0.706651939292811
10000000000.0: 0.704143836713699 

log regression algorithm plot created



In [47]:
svm_estimation = get_params_dic(svm_pred, cs_svm, X, y)
print('svm F1-score: ' + '\n', '\n'.join(['{}: {}'.format(key, svm_estimation[key]) for key in svm_estimation]), '\n')
create_plot('svm algorithm', svm_estimation)

/home/sokureo/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/sokureo/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


svm F1-score: 
 0.01: 0.26726750715874215
0.03: 0.26726750715874215
0.1: 0.5594647196724212
0.3: 0.6005673230638545
1: 0.6332864017809111
3: 0.6708481675662958
10: 0.6950401644317002
30: 0.7101924809337417
60: 0.7127655517963304
90: 0.7136350618242563 

svm algorithm plot created

